In [ ]:
# This is an open source project 

import pandas as pd
import numpy as np

# Simple Recommender


In [ ]:
metadata=pd.read_csv(r'C:\Users\Nancy Meshram\Downloads\archive (3)\movies_metadata.csv',low_memory=False)
metadata.head()

In [ ]:
# calculating mean of vote-average column

C=metadata['vote_average'].mean()
print(C)

In [ ]:
# calculating minimum no. of votes required for a movie to be in a chart
m=metadata['vote_count'].quantile(0.90)
print(m)

In [ ]:
# filtering out all the movies qualified into a new dataframe
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

In [ ]:
q_movies.head()

In [ ]:
metadata.shape

In [ ]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Defining a new feature 'score' and calculating its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
#Sorting movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Printing the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

# Content Based Recommender

In [ ]:
metadata['overview'].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')
metadata['overview']=metadata['overview'].fillna('')
tfidf_matrix=tfidf.fit_transform(metadata['overview'])
tfidf_matrix.shape

In [ ]:
tfidf_matrix

In [ ]:
tfidf.get_feature_names()[5000:5010]

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()
indices[:10]

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [ ]:
get_recommendations('Batman')